In [2]:
#!pip install lightgbm

  Using cached https://files.pythonhosted.org/packages/0b/9d/ddcb2f43aca194987f1a99e27edf41cf9bc39ea750c3371c2a62698c509a/lightgbm-2.3.1-py2.py3-none-manylinux1_x86_64.whl
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
from datetime  import datetime  
from datetime import timedelta  
import lightgbm as lgb
from sklearn import preprocessing, metrics


In [4]:
X_train = pd.read_pickle('../01_preprocessed_data/X_train.pkl')
y_train = pd.read_pickle('../01_preprocessed_data/y_train.pkl')
X_val = pd.read_pickle('../01_preprocessed_data/X_val.pkl')
y_val = pd.read_pickle('../01_preprocessed_data/y_val.pkl')
X_test = pd.read_pickle('../01_preprocessed_data/X_test.pkl')
submission = pd.read_csv('../00_data/sample_submission.csv')

In [5]:
#X_train = feather.read_dataframe('../01_preprocessed_data/X_train.feather')
#y_train = feather.read_dataframe('../01_preprocessed_data/y_train.feather')
#X_val = feather.read_dataframe('../01_preprocessed_data/X_val.feather')
#y_val = feather.read_dataframe('../01_preprocessed_data/y_val.feather')
#X_test = feather.read_dataframe('../01_preprocessed_data/X_test.feather')
#submission = pd.read_csv('../00_data/sample_submission.csv')

In [6]:
#X_train = X_train.drop(columns = ['date', 'start_date'])
#X_val = X_val.drop(columns = ['date', 'start_date'])
#X_test = X_test.drop(columns = ['date', 'start_date'])

In [7]:
#cat = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
#for feature in cat:
#    encoder = preprocessing.LabelEncoder()
#    X_train[feature] = encoder.fit_transform(X_train[feature])
#    X_val[feature] = encoder.fit_transform(X_val[feature])
#    X_test[feature] = encoder.fit_transform(X_test[feature])

In [8]:
#X_train = X_train.drop(columns = ['avg_month'])
#X_val = X_val.drop(columns = ['avg_month'])
#X_test= X_test.drop(columns = ['avg_month'])

In [9]:
X_test.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
       'items_sold', 'date', 'weekday', 'month', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI', 'start_date', 'days_from_start', 'start_date_from_start',
       'sell_price', 'avg_weekday_demand', 'avg_month_demand', 'avg_demand',
       'median_weekday_demand', 'median_month_demand', 'median_demand',
       'max_demand', 'avg_price', 'cheaper_than_usual', 'lag_t28', 'lag_t29',
       'lag_t30'],
      dtype='object')

In [14]:
params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective': 'regression',
        'n_jobs': -1,
        'seed': 0,
        'learning_rate': 0.1,
        'bagging_fraction': 0.75,
        'bagging_freq': 10, 
        'colsample_bytree': 0.75}
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id',
       'weekday', 'month', 'event_name_1',
       'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
       'snap_WI', 'days_from_start', 'start_date_from_start',
       'sell_price', 'avg_weekday_demand', 'avg_month_demand', 'avg_demand',
       'median_weekday_demand', 'median_month_demand', 'median_demand',
       'max_demand', 'avg_price', 'cheaper_than_usual', 'lag_t28', 'lag_t29',
       'lag_t30']
train_set = lgb.Dataset(X_train[features], y_train)
val_set = lgb.Dataset(X_val[features], y_val)

In [15]:
X_train[features].columns

Index(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
       'month', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'days_from_start',
       'start_date_from_start', 'sell_price', 'avg_weekday_demand',
       'avg_month_demand', 'avg_demand', 'median_weekday_demand',
       'median_month_demand', 'median_demand', 'max_demand', 'avg_price',
       'cheaper_than_usual', 'lag_t28', 'lag_t29', 'lag_t30'],
      dtype='object')

In [16]:
#del x_train, y_train

model = lgb.train(params, train_set, num_boost_round = 2500, early_stopping_rounds = 50, 
                  valid_sets = [train_set, val_set], verbose_eval = 100)

Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2.54144	valid_1's rmse: 2.21872
[200]	training's rmse: 2.49374	valid_1's rmse: 2.20696
[300]	training's rmse: 2.46592	valid_1's rmse: 2.20106
[400]	training's rmse: 2.44823	valid_1's rmse: 2.20008
Early stopping, best iteration is:
[367]	training's rmse: 2.45278	valid_1's rmse: 2.19842


In [17]:
val_pred = model.predict(X_val[features])
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
print(f'Our val rmse score is {val_score}')
y_test = model.predict(X_test[features])
X_test['items_sold'] = y_test


Our val rmse score is 2.198422624205129


In [18]:
X_test['items_sold'] = y_test
X_test

,id,item_id,dept_id,cat_id,store_id,state_id,items_sold,date,weekday,month,...,avg_demand,median_weekday_demand,median_month_demand,median_demand,max_demand,avg_price,cheaper_than_usual,lag_t28,lag_t29,lag_t30
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0.705917,2016-04-25,1,4,...,0.589844,0.0,0.0,0.589844,5,8.281250,False,1.0,1.0,1.0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,0.314727,2016-04-25,1,4,...,0.278076,0.0,0.0,0.278076,5,3.970703,False,1.0,1.0,1.0
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,0.320465,2016-04-25,1,4,...,0.353760,0.0,0.0,0.353760,6,2.970703,False,0.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1.847870,2016-04-25,1,4,...,1.750977,1.0,1.0,1.750977,15,4.523438,False,0.0,6.0,6.0
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,0.817035,2016-04-25,1,4,...,1.026367,0.0,1.0,1.026367,9,2.943359,True,1.0,0.0,0.0
5,HOBBIES_1_006_CA_1_validation,1442,3,1,0,0,1.002430,2016-04-25,1,4,...,1.104492,0.0,0.0,1.104492,10,0.991211,True,0.0,2.0,1.0
6,HOBBIES_1_007_CA_1_validation,1443,3,1,0,0,0.318862,2016-04-25,1,4,...,0.305420,0.0,0.0,0.305420,3,7.882812,True,1.0,0.0,0.0
7,HOBBIES_1_008_CA_1_validation,1444,3,1,0,0,3.959091,2016-04-25,1,4,...,7.230469,4.0,5.0,7.230469,91,0.476074,False,0.0,0.0,0.0
8,HOBBIES_1_009_CA_1_validation,1445,3,1,0,0,0.846944,2016-04-25,1,4,...,1.186523,0.0,1.0,1.186523,20,1.763672,False,0.0,0.0,0.0
9,HOBBIES_1_010_CA_1_validation,1446,3,1,0,0,0.542183,2016-04-25,1,4,...,0.719238,0.0,0.0,0.719238,6,2.982422,True,0.0,0.0,0.0


In [19]:
#submission = pd.read_csv('../00_data/sample_submission.csv')

In [20]:
X_test

,id,item_id,dept_id,cat_id,store_id,state_id,items_sold,date,weekday,month,...,avg_demand,median_weekday_demand,median_month_demand,median_demand,max_demand,avg_price,cheaper_than_usual,lag_t28,lag_t29,lag_t30
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,0.705917,2016-04-25,1,4,...,0.589844,0.0,0.0,0.589844,5,8.281250,False,1.0,1.0,1.0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,0.314727,2016-04-25,1,4,...,0.278076,0.0,0.0,0.278076,5,3.970703,False,1.0,1.0,1.0
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,0.320465,2016-04-25,1,4,...,0.353760,0.0,0.0,0.353760,6,2.970703,False,0.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1.847870,2016-04-25,1,4,...,1.750977,1.0,1.0,1.750977,15,4.523438,False,0.0,6.0,6.0
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,0.817035,2016-04-25,1,4,...,1.026367,0.0,1.0,1.026367,9,2.943359,True,1.0,0.0,0.0
5,HOBBIES_1_006_CA_1_validation,1442,3,1,0,0,1.002430,2016-04-25,1,4,...,1.104492,0.0,0.0,1.104492,10,0.991211,True,0.0,2.0,1.0
6,HOBBIES_1_007_CA_1_validation,1443,3,1,0,0,0.318862,2016-04-25,1,4,...,0.305420,0.0,0.0,0.305420,3,7.882812,True,1.0,0.0,0.0
7,HOBBIES_1_008_CA_1_validation,1444,3,1,0,0,3.959091,2016-04-25,1,4,...,7.230469,4.0,5.0,7.230469,91,0.476074,False,0.0,0.0,0.0
8,HOBBIES_1_009_CA_1_validation,1445,3,1,0,0,0.846944,2016-04-25,1,4,...,1.186523,0.0,1.0,1.186523,20,1.763672,False,0.0,0.0,0.0
9,HOBBIES_1_010_CA_1_validation,1446,3,1,0,0,0.542183,2016-04-25,1,4,...,0.719238,0.0,0.0,0.719238,6,2.982422,True,0.0,0.0,0.0


In [21]:
predictions = X_test[['id', 'date', 'items_sold']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'items_sold').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]

validation = submission[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])
final

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.705917,0.643470,0.495408,0.444049,0.553466,0.672205,0.372133,0.276388,0.541365,...,0.399097,0.663501,0.507192,0.400437,0.360641,0.326246,0.386434,0.453192,0.625731,0.419057
1,HOBBIES_1_002_CA_1_validation,0.314727,0.223475,0.159892,0.108533,0.118204,0.135748,0.254351,0.229439,0.230382,...,0.234376,0.249388,0.252371,0.212579,0.199874,0.293827,0.275547,0.291514,0.241948,0.241948
2,HOBBIES_1_003_CA_1_validation,0.320465,0.234834,0.234834,0.240241,0.297892,0.415979,0.404492,0.336753,0.314428,...,0.547945,0.698676,0.590799,0.473276,0.447968,0.421204,0.352910,0.451275,0.568606,0.511842
3,HOBBIES_1_004_CA_1_validation,1.847870,1.204663,0.817202,0.928952,1.508923,2.054259,2.612171,1.577528,1.560882,...,1.477140,1.738221,2.672594,1.901495,1.591584,1.271522,1.304906,1.619040,2.997567,2.993856
4,HOBBIES_1_005_CA_1_validation,0.817035,0.671193,0.954835,1.079979,1.101981,1.303462,1.397830,1.066081,1.010168,...,1.180006,1.365970,1.310994,0.865158,0.839242,0.890327,1.000567,1.178931,1.472312,1.535260
5,HOBBIES_1_006_CA_1_validation,1.002430,0.907522,0.910699,0.893680,0.793251,0.828205,0.985115,0.937894,0.865279,...,0.840653,1.220625,1.102059,1.182424,0.928988,0.860723,0.835920,1.025218,1.286470,1.141637
6,HOBBIES_1_007_CA_1_validation,0.318862,0.388273,0.369377,0.341646,0.330687,0.340884,0.440333,0.332246,0.327374,...,0.337183,0.368259,0.371242,0.409340,0.367131,0.447586,0.372539,0.394321,0.454773,0.521128
7,HOBBIES_1_008_CA_1_validation,3.959091,6.242989,6.582724,6.020555,7.488158,11.403844,9.490267,7.582957,5.872889,...,5.857229,5.024736,3.862704,7.296928,5.800493,6.050619,5.149109,5.471510,6.097018,4.749808
8,HOBBIES_1_009_CA_1_validation,0.846944,1.153362,0.957443,1.168501,1.224181,1.488488,1.542880,1.048861,1.091067,...,0.836741,1.048109,1.161763,1.079588,1.483880,1.223827,1.174866,0.815310,1.042216,0.940698
9,HOBBIES_1_010_CA_1_validation,0.542183,0.617616,0.699809,0.755300,0.794535,0.798164,1.040776,0.761409,0.818081,...,0.628594,0.673298,0.771664,0.502485,0.483964,0.483964,0.489371,0.651348,0.702684,1.040386


In [27]:

final.to_csv('../04_submissions/lightGBM_no_fe.csv', index = False)

In [28]:
features.append(str(val_score))

In [29]:
with open("../04_submissions/lgb_features_score.txt", "a") as outfile:
    outfile.write("\n".join(features))
